### 並行処理と並列処理

### 逐次処理で実行

### 並行処理で実行する

### 並列処理で実行する

### Pythonと並行処理

### 並行処理と非同期処理の関係

### concurrent.futuresモジュール　　　並行処理のための高水準インタフェース

### FutureクラスとExecutorクラス　　　非同期処理のカプセル化と実行

In [1]:
# ThreadPoolExecutorはExecutorの具象サブクラス
from concurrent.futures import (
    ThreadPoolExecutor,
    Future
)

In [2]:
# 非同期に行いたい処理
def func():
    return 1

In [3]:
# 非同期に行いたい処理をsubmit()に渡す
future = ThreadPoolExecutor().submit(func)
isinstance(future, Future)

True

In [4]:
# 非同期で実行した処理の戻り値を取得
future.result()

1

In [5]:
# 現在の状態を確認する
future.done()

True

In [6]:
future.running()

False

In [7]:
future.cancelled()

False

### ThreadPoolExecutorクラス　　　スレッドベースの非同期実行

### スレッドベースの非同期実行が効果的なケース

### ThreadPoolExecutorクラスを利用したマルチスレッド処理の実例

In [7]:
# 対象ページのURL一覧
urls = [
    'https://twitter.com',
    'https://facebook.com',
    'https://instagram.com'
]

In [8]:
from hashlib import md5
from pathlib import Path
from urllib import request

In [9]:
def download(url):
    req = request.Request(url)
    # ファイrう名に/などが含まれないようにする
    name = md5(url.encode('utf-8')).hexdigest()
    file_path = './' + name
    with request.urlopen(req) as res:
        Path(file_path).write_bytes(res.read())
        return url, file_path

In [10]:
# 動きを確認
download(urls[0])

('https://twitter.com', './be8b09f7f1f66235a9c91986952483f0')

### 逐次処理で実装

In [11]:
import time
def elapsed_time(f):
    def wrapper(*args, **kwargs):
        st = time.time()
        v = f(*args, **kwargs)
        print(f"{f.__name__}: {time.time() - st}")
        return v
    return wrapper

In [12]:
@elapsed_time
def get_sequential():
    for url in urls:
        print(download(url))

In [13]:
get_sequential()

('https://twitter.com', './be8b09f7f1f66235a9c91986952483f0')
('https://facebook.com', './a023cfbf5f1c39bdf8407f28b60cd134')
('https://instagram.com', './09f8b89478d7e1046fa93c7ee4afa99e')
get_sequential: 2.032212495803833


### マルチスレッドで実装

In [15]:
from concurrent.futures import (
    ThreadPoolExecutor,
    as_completed
)

In [16]:
@elapsed_time
def get_multi_thread():
    # max_workersのデフォルトはコア数x5
    with ThreadPoolExecutor(max_workers=3) as executor:
        futures = [executor.submit(download, url)
                   for url in urls]
        for future in as_completed(futures):
            # 完了したものから取得できる
            print(future.result())

In [17]:
get_multi_thread()

('https://twitter.com', './be8b09f7f1f66235a9c91986952483f0')
('https://facebook.com', './a023cfbf5f1c39bdf8407f28b60cd134')
('https://instagram.com', './09f8b89478d7e1046fa93c7ee4afa99e')
get_multi_thread: 1.2520246505737305
